In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn import datasets, model_selection, svm
from src.PCapLocationParams import PCapLocationParams
import matplotlib
from matplotlib import pyplot
import sqlite3 as sql
import src.view.DataView as dview
import pyarrow

In [2]:
pd.options.display.width = 920
pd.options.display.max_columns = 500
pd.options.display.max_rows = 220

In [3]:
pcap = PCapLocationParams(340, '2021-06-09')
secs = pd.read_feather(pcap.feather_file_path('security'))
synth_leg_bks = pd.read_feather(pcap.feather_file_path('synthetic_leg_book'))
synth_leg_bbo = pd.read_feather(pcap.feather_file_path('synthetic_leg_book_2021-07-02_05-26'))
header_cols = ['eid', 'sid', 'symbol', 'asset', 'sec_group', 't_time', 'status']
synth_leg_bks = pd.merge(synth_leg_bks, secs[['sid', 'asset', 'sec_group', 'symbol']], on='sid')
synth_leg_bbo = pd.merge(synth_leg_bbo, secs[['sid', 'asset', 'sec_group', 'symbol']], on='sid')
slb = synth_leg_bbo.loc[synth_leg_bbo['sid'] == 24491].copy()

In [61]:
# slb.rename(columns={'bid_p_1': 'bid'})
# slb.set_index('t_time', inplace=True)
# slb.sort_index(inplace=True)
def get_window_bbo_bounds(win):
    return win.max()
#     max_bid_idx = win['bid_p_1'].idxmax()
#     max_bid_idx = win['bid_p_1'].idxmax()
#     return [win['bid_p_1'].min(), win['bid_p_1'].max(), win['ask_p_1'].max(), win['ask_p_1'].min()]
#     return 0

win_sz = '5s'
slb[f'bid_pmax_1_{win_sz}'] = slb['bid_p_1'].rolling(win_sz).max()
slb[f'bid_pmin_1_{win_sz}'] = slb['bid_p_1'].rolling(win_sz).min()
slb[f'ask_pmax_1_{win_sz}'] = slb['ask_p_1'].rolling(win_sz).max()
slb[f'ask_pmin_1_{win_sz}'] = slb['ask_p_1'].rolling(win_sz).min()
slb[f'qarb_{win_sz}'] = (slb[f'bid_pmax_1_{win_sz}']) - (slb[f'ask_pmin_1_{win_sz}'])
slb[f'barb_{win_sz}'] = (slb[f'bid_pmin_1_{win_sz}']) - (slb[f'ask_pmax_1_{win_sz}'])
qarbs = slb.loc[slb[f'qarb_{win_sz}'] > 0]
barbs = slb.loc[slb[f'qarb_{win_sz}'] < 0]
# slb.describe()
# slb.loc[(slb[f'bid_pmax_1_{win_sz}']) > (slb[f'ask_pmin_1_{win_sz}'])]

In [62]:
# qarbs.reset_index().drop_duplicates(keep='last')
qarbs['is_dup'] = qarbs.index.duplicated()
qarbs.head(10)
# qarbs.style.apply(dview.get_style_dict(qarbs))
# barbs.describe()

<ipython-input-62-3a401089220d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qarbs['is_dup'] = qarbs.index.duplicated()


,eid,sid,symbol,asset,sec_group,status,bid_q_1,bid_p_1,ask_p_1,ask_q_1,ibid_q_1,ibid_p_1,iask_p_1,iask_q_1,bid_pmax_1_5s,bid_pmin_1_5s,ask_pmax_1_5s,ask_pmin_1_5s,qarb_5s,barb_5s,is_dup
t_time,,,,,,,,,,,,,,,,,,,,,
2021-06-10 16:00:41.087722309,14645506,24491,ZSN1-ZSQ1,ZS,ZS,READY_TO_TRADE,59,38.0,38.25,2,2,36.75,39.00,2,38.5,38.0,38.75,38.25,0.25,-0.75,False
2021-06-10 16:00:41.088044953,14645510,24491,ZSN1-ZSQ1,ZS,ZS,READY_TO_TRADE,59,38.0,38.25,2,2,36.75,39.00,2,38.5,38.0,38.75,38.25,0.25,-0.75,False
2021-06-10 16:00:41.088344209,14645563,24491,ZSN1-ZSQ1,ZS,ZS,READY_TO_TRADE,59,38.0,38.25,2,2,36.75,39.00,2,38.5,38.0,38.75,38.25,0.25,-0.75,False
2021-06-10 16:00:41.093226855,14645722,24491,ZSN1-ZSQ1,ZS,ZS,READY_TO_TRADE,59,38.0,38.25,2,2,36.75,39.00,2,38.5,38.0,38.75,38.25,0.25,-0.75,False
2021-06-10 16:00:41.093834587,14645745,24491,ZSN1-ZSQ1,ZS,ZS,READY_TO_TRADE,60,38.0,38.25,2,2,36.75,39.00,2,38.5,38.0,38.75,38.25,0.25,-0.75,False
2021-06-10 16:00:41.107365555,14645911,24491,ZSN1-ZSQ1,ZS,ZS,READY_TO_TRADE,60,38.0,38.25,2,2,36.75,39.00,3,38.5,38.0,38.75,38.25,0.25,-0.75,False
2021-06-10 16:00:41.107476117,14645914,24491,ZSN1-ZSQ1,ZS,ZS,READY_TO_TRADE,60,38.0,38.25,2,2,36.75,38.75,2,38.5,38.0,38.75,38.25,0.25,-0.75,False
2021-06-10 16:00:41.133336095,14646093,24491,ZSN1-ZSQ1,ZS,ZS,READY_TO_TRADE,61,38.0,38.25,2,2,36.75,38.75,2,38.5,38.0,38.75,38.25,0.25,-0.75,False
2021-06-10 16:00:41.146477913,14646195,24491,ZSN1-ZSQ1,ZS,ZS,READY_TO_TRADE,60,38.0,38.25,2,2,36.75,38.75,2,38.5,38.0,38.75,38.25,0.25,-0.75,False


In [18]:
slb['bid_p_1'].rolling(window='5us').apply(get_window_bbo_bounds)

t_time
2021-06-09 21:39:59.568501189    38.75
2021-06-09 21:45:00.000000000    38.75
2021-06-09 21:55:00.926006505    38.75
2021-06-09 21:55:01.632204373    38.75
2021-06-09 21:58:32.128764345    38.75
                                 ...  
2021-06-10 18:30:00.070403705    33.25
2021-06-10 19:30:00.000000000    33.25
2021-06-10 19:46:59.622728075    33.25
2021-06-10 20:32:39.817111007    34.00
2021-06-10 21:00:00.000000000    34.00
Name: bid_p_1, Length: 478597, dtype: float64